<a href="https://colab.research.google.com/github/bryaanabraham/Wandering_Guardian/blob/main/ML%20Implementation/ML_Navigation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import folium
import random
from geopy.distance import geodesic

In [142]:
coords = pd.read_csv("training_dataset.csv")
coords.head()

,longitude,latitude,path_id
0,12.821062,80.044729,1
1,12.821219,80.044743,1
2,12.821479,80.044736,1
3,12.821561,80.044743,1
4,12.821636,80.044743,1


In [143]:
path = []
for i in range(len(coords[' path_id'].unique())):
    path.append(coords[coords[' path_id'] == i])

In [144]:
path[1].head()

,longitude,latitude,path_id
0,12.821062,80.044729,1
1,12.821219,80.044743,1
2,12.821479,80.044736,1
3,12.821561,80.044743,1
4,12.821636,80.044743,1


In [145]:
# Deviation parameters
min_deviation_distance = -0.00004
max_deviation_distance = 0.00004
#deviation is set to zero for now so elevation remains constant
max_deviation_angle = 0

# Generate random deviation distances and angles for each point in the DataFrame
num_points = len(coords)
random_deviation_distances = np.random.uniform(min_deviation_distance, max_deviation_distance, num_points)
random_deviation_angles = np.random.uniform(0, max_deviation_angle, num_points)

# Convert random angles to radians
random_deviation_angles_radians = np.radians(random_deviation_angles)

# Calculate deviation in latitude and longitudet
deviation_distance_lat = random_deviation_distances * np.cos(random_deviation_angles_radians)
deviation_distance_lon = random_deviation_distances * np.sin(random_deviation_angles_radians)

decoy_coords = coords.copy()
decoy_coords[' latitude'] += deviation_distance_lat
decoy_coords['longitude'] += deviation_distance_lon

decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044739,1
1,12.821219,80.044774,1
2,12.821479,80.044703,1
3,12.821561,80.044769,1
4,12.821636,80.044735,1


In [146]:
decoy_coords.head()

,longitude,latitude,path_id
0,12.821062,80.044739,1
1,12.821219,80.044774,1
2,12.821479,80.044703,1
3,12.821561,80.044769,1
4,12.821636,80.044735,1


In [147]:
print('coords keys:', coords.keys())
print('decoy_coords keys:', decoy_coords.keys())

coords keys: Index(['longitude', ' latitude', ' path_id'], dtype='object')
decoy_coords keys: Index(['longitude', ' latitude', ' path_id'], dtype='object')


In [148]:
decoy_path = []
for i in range(len(decoy_coords[' path_id'].unique())):
    decoy_path.append(decoy_coords[decoy_coords[' path_id'] == i])

In [149]:
maps = []

colors = ['red','blue','green','yellow','orange','purple','pink','cyan','black','brown','maroon','navy','teal','olive','lime','aqua','silver','gold']

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords[' latitude'])
max_lon = max(coords[' latitude'])

n = len(coords[' path_id'].unique())

#each path_id has its own map and colour
for i in range(1,n):

  m = folium.Map()
  color_path = colors[i]
  gps_path = list(zip(path[i]['longitude'], path[i][' latitude']))
  folium.PolyLine(locations=gps_path, color=color_path, weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_path[20], icon=folium.Icon(color=color_path), popup=f'Safe Path').add_to(m)

  color_decoy_path = colors[i + n]
  gps_decoy_path = list(zip(decoy_path[i]['longitude'], decoy_path[i][' latitude']))
  folium.PolyLine(locations=gps_decoy_path, color=color_decoy_path, weight=2.5, opacity=0.8).add_to(m)
  folium.Marker(gps_decoy_path[10], icon=folium.Icon(color=color_decoy_path), popup=f'Decoy Path').add_to(m)

  m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])
  maps.append(m)

  display(maps[i-1])


<ipython-input-149-d3513860ba92>:27: UserWarning: color argument of Icon should be one of: {'darkred', 'purple', 'darkblue', 'darkgreen', 'blue', 'lightblue', 'lightred', 'red', 'cadetblue', 'gray', 'pink', 'orange', 'lightgray', 'white', 'darkpurple', 'beige', 'black', 'lightgreen', 'green'}.
  folium.Marker(gps_decoy_path[10], icon=folium.Icon(color=color_decoy_path), popup=f'Decoy Path').add_to(m)


<ipython-input-149-d3513860ba92>:22: UserWarning: color argument of Icon should be one of: {'darkred', 'purple', 'darkblue', 'darkgreen', 'blue', 'lightblue', 'lightred', 'red', 'cadetblue', 'gray', 'pink', 'orange', 'lightgray', 'white', 'darkpurple', 'beige', 'black', 'lightgreen', 'green'}.
  folium.Marker(gps_path[20], icon=folium.Icon(color=color_path), popup=f'Safe Path').add_to(m)


In [152]:
def calculate_deviation(coord1, coord2):
    return geodesic(coord1, coord2).kilometers

In [150]:
m = folium.Map()

gps_path = list(zip(decoy_coords['longitude'], decoy_coords[' latitude']))
folium.PolyLine(locations=gps_path, color='red', weight=2.5, opacity=0.8).add_to(m)

gps_path = list(zip(coords['longitude'], coords[' latitude']))
folium.PolyLine(locations=gps_path, color='blue', weight=2.5, opacity=0.8).add_to(m)

max_lat = max(coords['longitude'])
min_lat = min(coords['longitude'])
min_lon = min(coords[' latitude'])
max_lon = max(coords[' latitude'])
m.fit_bounds([(min_lat, min_lon), (max_lat, max_lon)])

deviations = []

for i in range(0, min(len(coords), len(decoy_coords)), 5):
    deviation = calculate_deviation((coords.iloc[i]['longitude'], coords.iloc[i][' latitude']),
                                  (decoy_coords.iloc[i]['longitude'], decoy_coords.iloc[i][' latitude']))
    deviations.append((i, deviation*1000))

for index, deviation in deviations:
    coord = (decoy_coords.iloc[index]['longitude'], decoy_coords.iloc[index][' latitude'])
    if deviation <= 2:
      colour = 'green'
    elif deviation >= 4:
      colour = 'red'
    else:
      colour = 'orange'
    m.add_child(folium.Marker(location=coord,
                              popup=f"Deviation: {deviation:.2f} m",
                              icon=folium.Icon(color=colour)))
m